### Import required libraries

In [1]:
from tqdm import tqdm
import os
import json
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text  # A dependency of the preprocessing model
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np

2023-04-05 10:09:50.424307: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-05 10:09:50.965204: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-05 10:09:50.968436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 10:09:52.699515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/sylvas/anaconda3/envs/tensorflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sylvas/anacon

##### Download required library

In [149]:
# # Install the following libraries
# !pip install spacy
# !pip install gensim
# !pip install pyldavis
# !pip install wordcloud
# !pip install textblob
# !pip install textstat
# !pip install nltk
# !pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 1.5 MB/s eta 0:00:00m eta 0:00:010:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.0/911.0 kB 2.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 6.4 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 2.1 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.5 MB/s eta 0:00:00

In [151]:
# Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import nltk, time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
import collections, itertools

## Data Wrangling

##### Convertion function to extract data from json files into single csv file

In [2]:
# @title Function to extract data from json files into single csv file
def get_data_to_csv(src_path='../datasets/ecir/EN_train', out_path='../datasets/ecir/ecir_csv'):
    path_no = 0

    paths = ['../datasets/ecir/EN_train', '../datasets/ecir/EN_dev', '../datasets/ecir/EN_test']
    for case_path in paths:
        total = 0
        for item in tqdm(os.listdir(case_path)):
            if item.endswith('.json'):
                # print(item)
                with open(os.path.join(case_path, item), "r") as json_file:
                    data = json.load(json_file)
                    # data = json.dumps(data,indent=2)
                    if total == 0:
                        df = pd.DataFrame([data])
                    # elif total == 2:
                    #     break
                    else:
                        data = pd.DataFrame([data])
                        df = pd.concat([df,data], axis=0)
                    # df.head()
                json_file.close()
                
            total += 1
            
        if path_no == 0:
            df.to_csv('../datasets/ecir/ecir_csv/train.csv')
        elif path_no == 1:
            df.to_csv('../datasets/ecir/ecir_csv/dev.csv')
        else:
            df.to_csv('../datasets/ecir/ecir_csv/test.csv')
                    # break
        path_no += 1

    return df.head()

In [9]:
test = pd.read_csv('../datasets/ecir_csv/test.csv')
train = pd.read_csv('../datasets/ecir_csv/train.csv')
dev = pd.read_csv('../datasets/ecir_csv/dev.csv')
test.head()

,Unnamed: 0,ITEMID,LANGUAGEISOCODE,RESPONDENT,BRANCH,DATE,DOCNAME,IMPORTANCE,CONCLUSION,JUDGES,TEXT,VIOLATED_ARTICLES,VIOLATED_PARAGRAPHS,VIOLATED_BULLETPOINTS,NON_VIOLATED_ARTICLES,NON_VIOLATED_PARAGRAPHS,NON_VIOLATED_BULLETPOINTS
0,0,001-147041,ENG,RUS,COMMITTEE,2014,CASE OF MAKOVOZ v. RUSSIA,4,Violation of Article 3 - Prohibition of tortur...,Dmitry Dedov;Julia Laffranque;Khanlar Hajiyev,['4. The applicant was born in 1968 and lived ...,['3'],[],[],[],[],[]
1,0,001-172665,ENG,SMR,CHAMBER,2017,CASE OF PODESCHI v. SAN MARINO,3,No violation of Article 3 - Prohibition of tor...,Aleš Pejchal;Kristina Pardalos;Ledi Bianku;Lin...,['5. The applicant was born in 1956 and lives ...,[],[],[],"['3', '5']","['5-3', '5-4']",[]
2,0,001-148075,ENG,NLD,ADMISSIBILITY,2014,ÖZTÜRK v. THE NETHERLANDS,4,Inadmissible,Alvina Gyulumyan;Ján Šikuta;Johannes Silvis;Jo...,"['1. The applicant, Mr Fuat Öztürk, is a Turki...",[],[],[],[],[],[]
3,0,001-166482,ENG,TUR,CHAMBER,2016,CASE OF TOPTANIŞ v. TURKEY,4,Violation of Article 2 - Right to life (Articl...,Jon Fridrik Kjølbro;Julia Laffranque;Nebojša V...,['4. The applicant was born in 1974 and lives ...,['2'],['2-1'],[],[],[],[]
4,0,001-169211,ENG,UKR,COMMITTEE,2016,CASE OF GAVRASHENKO AND OTHERS v. UKRAINE,4,Violation of Article 6 - Right to a fair trial...,Carlo Ranzoni;Khanlar Hajiyev,['3. The list of applicants and the relevant d...,"['13', '6', 'P4']","['6-1', 'P4-2']",['P4-2-1'],[],[],[]


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 7100 non-null   int64 
 1   ITEMID                     7100 non-null   object
 2   LANGUAGEISOCODE            7100 non-null   object
 3   RESPONDENT                 7100 non-null   object
 4   BRANCH                     7100 non-null   object
 5   DATE                       7100 non-null   int64 
 6   DOCNAME                    7100 non-null   object
 7   IMPORTANCE                 7100 non-null   int64 
 8   CONCLUSION                 7100 non-null   object
 9   JUDGES                     6383 non-null   object
 10  TEXT                       7100 non-null   object
 11  VIOLATED_ARTICLES          7100 non-null   object
 12  VIOLATED_PARAGRAPHS        7100 non-null   object
 13  VIOLATED_BULLETPOINTS      7100 non-null   object
 14  NON_VIOL

In [13]:
train.columns

Index(['Unnamed: 0', 'ITEMID', 'LANGUAGEISOCODE', 'RESPONDENT', 'BRANCH',
       'DATE', 'DOCNAME', 'IMPORTANCE', 'CONCLUSION', 'JUDGES', 'TEXT',
       'VIOLATED_ARTICLES', 'VIOLATED_PARAGRAPHS', 'VIOLATED_BULLETPOINTS',
       'NON_VIOLATED_ARTICLES', 'NON_VIOLATED_PARAGRAPHS',
       'NON_VIOLATED_BULLETPOINTS'],
      dtype='object')

In [130]:
new_train = train.iloc[:,7:13]
new_test = test.iloc[:,7:13]
new_dev = dev.iloc[:,7:13]
new_train.head(2)

,IMPORTANCE,CONCLUSION,JUDGES,TEXT,VIOLATED_ARTICLES,VIOLATED_PARAGRAPHS
0,3,Preliminary objection rejected (six month peri...,C. Russo;N. Valticos,['6. On 23 December 1987 Mr Papageorgiou and 1...,['6'],['6-1']
1,4,Inadmissible,David Thór Björgvinsson;Ján Šikuta;Ledi Bianku...,['The applicants are the following six private...,[],[]


In [131]:
new_train.head(7)

,IMPORTANCE,CONCLUSION,JUDGES,TEXT,VIOLATED_ARTICLES,VIOLATED_PARAGRAPHS
0,3,Preliminary objection rejected (six month peri...,C. Russo;N. Valticos,['6. On 23 December 1987 Mr Papageorgiou and 1...,['6'],['6-1']
1,4,Inadmissible,David Thór Björgvinsson;Ján Šikuta;Ledi Bianku...,['The applicants are the following six private...,[],[]
2,2,Not necessary to examine Art. 2;Not necessary ...,Feyyaz Gölcüklü,['10. The applicants are six women from northe...,[],[]
3,4,Inadmissible,Nicolas Bratza,"['The applicant, Mr Ivan Angelov, is a Bulgari...",[],[]
4,3,No violation of Art. 8;No violation of Art. 14+8,NaN,['8. The applicant was born in 1974 and lives ...,[],[]
5,3,Preliminary objection dismissed (non-exhaustio...,Feyyaz Gölcüklü,"['8. The applicant, a Turkish citizen of Kurdi...","['13', '3', '8']",[]
6,3,Violation of Art. 6-1;Non-pecuniary damage - f...,Feyyaz Gölcüklü;N. Valticos;R. Pekkanen,['6. Mr Benkessiouer is a French national who ...,['6'],['6-1']


#### Strip data of excessess

In [132]:
def strip_data(data):
    # strip violated_articles values
    strip_data = [ str(x).replace("[]",'null').replace("'",'').strip("[]").strip() for x in data['VIOLATED_ARTICLES']]
    data['clean_violated_articles'] = strip_data
    # strip violated_paragraphs values
    strip_data = [ str(x).replace("[]",'null').replace("'",'').strip("[]").strip() for x in data['VIOLATED_PARAGRAPHS']]
    data['clean_violated_paragraphs'] = strip_data
    # strip text values
    strip_data = [ str(x).replace("['",'').replace("']",'').strip() for x in data['TEXT']]
    data['clean_text'] = strip_data
    # determined guilty or not base on the status of conc
    strip_data = [ 0 if x.strip() == 'null' else 1 for x in data['clean_violated_articles']]
    data['decision'] = strip_data
    # replace null values with
    data = data.replace('null',np.nan)
    data = data.drop(['TEXT','VIOLATED_PARAGRAPHS','VIOLATED_ARTICLES','CONCLUSION','JUDGES'], axis=1)

    return data

In [133]:
np.nan

nan

In [134]:
new_dev = strip_data(new_dev)
new_test = strip_data(new_test)
new_train = strip_data(new_train)

#### save clean data

In [141]:
new_dev.to_csv('../datasets/ecir_csv/clean_dev.csv')
new_test.to_csv('../datasets/ecir_csv/clean_test.csv')
new_train.to_csv('../datasets/ecir_csv/clean_train.csv')

In [136]:
new_train.head()

,IMPORTANCE,clean_violated_articles,clean_violated_paragraphs,clean_text,decision
0,3,6,6-1,6. On 23 December 1987 Mr Papageorgiou and 109...,1
1,4,NaN,NaN,The applicants are the following six private i...,0
2,2,NaN,NaN,10. The applicants are six women from northern...,0
3,4,NaN,NaN,"The applicant, Mr Ivan Angelov, is a Bulgarian...",0
4,3,NaN,NaN,8. The applicant was born in 1974 and lives in...,0


In [142]:
new_train['clean_violated_articles'].isna().sum()

690

In [145]:
train_text = new_train['clean_text']
train_text.head()

0    6. On 23 December 1987 Mr Papageorgiou and 109...
1    The applicants are the following six private i...
2    10. The applicants are six women from northern...
3    The applicant, Mr Ivan Angelov, is a Bulgarian...
4    8. The applicant was born in 1974 and lives in...
Name: clean_text, dtype: object

In [146]:
train_text[1]

"The applicants are the following six private insurance companies which were established under the laws of Slovakia in 1991, 1993, 1992, 1994, 1997 and 1991 respectively and have their registered seats in Bratislava (Slovakia): Allianz – Slovenská poisťovňa, a.s.; Česká poisťovňa - Slovensko, akciová spoločnosť; ČSOB Poisťovňa, a.s.; KOMUNÁLNA poisťovňa, a.s. Vienna Insurance Group; Generali Slovensko poisťovňa, a. s. and UNIQA poisťovňa, a.s.', 'The applicant companies were represented before the Court by Mr Ľ. Fogaš, a lawyer practising in Bratislava.', 'The Slovak Government (“the Government”) were represented by Ms M. Pirošíková, their Agent, and Ms M. Bálintová, Co-Agent.', 'The facts of the case, as submitted by the parties, may be summarised as follows.', 'On 1 March 2002 Act no. 95/2002 Coll. on Insurance (“the Act”) came into force. The Act set an obligation for insurance companies doing business in the field of compulsory contractual insurance of liability for damage caused b

### Exploratory Data Analysis on clean_text

In [158]:
train_text = new_train['clean_text']
test_text = new_test['clean_text']
dev_text = new_dev['clean_text']

In [152]:
# Function to tokenize the tweets
def custom_tokenize(text):
    """Function that tokenizes text"""
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)

In [155]:
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sylvas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Test tranformation

#### Step 1: Sentence segmentation

In [167]:
# #import spacy library
# import spacy

# #load core english library
# nlp = spacy.load("en_core_web_sm")

# #take unicode string
# #here u stands for unicode
# # doc = nlp(u"I Love Coding. Geeks for Geeks helped me in this regard very much. I Love Geeks for Geeks.")
# doc = nlp(dev_text[1])
# #to print sentences
# for sent in doc.sents:
#     print(sent)


The applicant, Ms Lyubov Ivanovna Bordokina, is a Ukrainian national who lives in Molodogvardiysk.
The Ukrainian Government (“the Government”) were represented by Mr Y. Zaytsev, their Agent, and Mrs I. Shevchuk, Head of the Office of the Government Agent before the European Court of Human Rights.', 'In July 1994 the applicant deposited 1,000,000 karbovantsi (around 30 ECU in the Ukrainian transitional currency) with the private commercial bank “Privatbank” (the “Bank”; Комерційний банк “Приватбанк”) for a four-year term at an annual interest rate of 500%.
At the end of the contract, the Bank paid the applicant 182 Ukrainian hryvnyas (about 0.70 ECU), referring to the introduction of the new hryvnya currency and a revision of interest rates as allowed by the contract between the parties.', 'On 30 September 1998 the applicant instituted civil proceedings in the Leninsky District Court of Lugansk (the “District Court”; Ленінський районний суд м.
Луганська) seeking higher payment and damag

In [169]:
doc = nltk.sent_tokenize(dev_text[1])
#to print sentences
for sent in doc:
    print(sent)

AttributeError: 'Series' object has no attribute 'encode'

#### Step 2: Word tokenization.

In [156]:
tokenized = train_text.apply(custom_tokenize) # Tokenize text
tokenized

0       [6, ., On, 23, December, 1987, Mr, Papageorgio...
1       [The, applicants, are, the, following, six, pr...
2       [10, ., The, applicants, are, six, women, from...
3       [The, applicant, ,, Mr, Ivan, Angelov, ,, is, ...
4       [8, ., The, applicant, was, born, in, 1974, an...
                              ...                        
7095    [The, applicant, ,, Mr, Tibor, Vrábel, ,, is, ...
7096    [4, ., The, applicant, was, born, in, 1972, .,...
7097    [4, ., The, applicant, ,, a, retired, military...
7098    [The, applicant, ,, a, Canadian, citizen, born...
7099    [6, ., All, the, applicants, were, held, in, v...
Name: clean_text, Length: 7100, dtype: object

In [157]:
tokenized[1]

['The',
 'applicants',
 'are',
 'the',
 'following',
 'six',
 'private',
 'insurance',
 'companies',
 'which',
 'were',
 'established',
 'under',
 'the',
 'laws',
 'of',
 'Slovakia',
 'in',
 '1991',
 ',',
 '1993',
 ',',
 '1992',
 ',',
 '1994',
 ',',
 '1997',
 'and',
 '1991',
 'respectively',
 'and',
 'have',
 'their',
 'registered',
 'seats',
 'in',
 'Bratislava',
 '(',
 'Slovakia',
 ')',
 ':',
 'Allianz',
 '–',
 'Slovenská',
 'poisťovňa',
 ',',
 'a.s.',
 ';',
 'Česká',
 'poisťovňa',
 '-',
 'Slovensko',
 ',',
 'akciová',
 'spoločnosť',
 ';',
 'ČSOB',
 'Poisťovňa',
 ',',
 'a.s.',
 ';',
 'KOMUNÁLNA',
 'poisťovňa',
 ',',
 'a.s.',
 'Vienna',
 'Insurance',
 'Group',
 ';',
 'Generali',
 'Slovensko',
 'poisťovňa',
 ',',
 'a.',
 's.',
 'and',
 'UNIQA',
 'poisťovňa',
 ',',
 'a.s.',
 "'",
 ',',
 "'The",
 'applicant',
 'companies',
 'were',
 'represented',
 'before',
 'the',
 'Court',
 'by',
 'Mr',
 'Ľ.',
 'Fogaš',
 ',',
 'a',
 'lawyer',
 'practising',
 'in',
 'Bratislava',
 '.',
 "'",
 ',',
 "'T